In [ ]:
from bayesbeat.result import get_fit
from bayesbeat.data import get_data, get_n_entries
import h5py
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pathlib

from utils import (
    get_duration,
    add_log10_bayes_factor_colorbar,
    get_bayes_factor_markers,
    get_frequency,
    get_duration,
)

plt.style.use("paper.mplstyle")

In [ ]:
outdir = pathlib.Path("figures")
outdir.mkdir(exist_ok=True)

file_format = "pdf"

Path to the data file

In [ ]:
data_file = "../data/PyTotalAnalysis_2024_02_23.mat"
n_ringdowns = get_n_entries(data_file)

In [ ]:
paths = {
    "simple_model_constant_noise": pathlib.Path("../analysis/gens_data/rundir/simple_model_constant_noise_fix/"),
    "simple_model_amp_noise": pathlib.Path("../analysis/gens_data/rundir/simple_model_both_noise_sources_fix/"),
    "terms_1": pathlib.Path("../analysis/gens_data/rundir/analytic_model_1_term_both_noise_sources/"),
    "terms_3": pathlib.Path("../analysis/gens_data/rundir/analytic_model_3_terms_both_noise_sources/"),
    "terms_5": pathlib.Path("../analysis/gens_data/rundir/analytic_model_5_terms_both_noise_sources/"),
    "terms_7": pathlib.Path("../analysis/gens_data/rundir/analytic_model_7_terms_both_noise_sources/"),
    # "terms_1_no_gap": pathlib.Path("../analysis/rundir/analytic_model_1_term_both_noise_sources_no_gap/"), 
    # "terms_3_no_gap": pathlib.Path("../analysis/rundir/analytic_model_3_terms_both_noise_sources_no_gap/"),
    # "terms_5_no_gap": pathlib.Path("../analysis/rundir/analytic_model_5_terms_both_noise_sources_no_gap/"),
    # "terms_7_no_gap": pathlib.Path("../analysis/rundir/analytic_model_7_terms_both_noise_sources_no_gap/"),
}

In [ ]:
nevals = {}
times = {}
for key, path in paths.items():
    nevals[key] = np.empty(n_ringdowns)
    times[key] = np.empty(n_ringdowns)
    for index in range(n_ringdowns):
        result_file=path / "analysis" / f"index_{index}" / "result.hdf5"
        with h5py.File(result_file, "r") as res_file:
            nevals[key][index] = res_file["total_likelihood_evaluations"][()]
            times[key][index] = res_file["sampling_time"][()]

In [ ]:
parameters = {}
for key, path in paths.items():
    d = dict(
        a_1=np.nan * np.empty(n_ringdowns),
        a_ratio=np.nan * np.empty(n_ringdowns),
        a_scale=np.nan * np.empty(n_ringdowns),
    )
    for index in range(n_ringdowns):
        result_file = path / "analysis" / f"index_{index}" / "result.hdf5"
        try:
            with h5py.File(result_file, "r") as res_file:
                posterior_samples = res_file["posterior_samples"][()]
            max_logl_idx = np.argmax(posterior_samples["logL"])
            for p in posterior_samples.dtype.names:
                if p not in d:
                    d[p] = np.empty(n_ringdowns)
                d[p][index] = posterior_samples[p][max_logl_idx]
        except FileNotFoundError:
            pass
    parameters[key] = d

In [ ]:
log_z = {}
info = {}
for key, path in paths.items():
    log_z[key] = np.empty(n_ringdowns)
    info[key] = np.empty(n_ringdowns)
    for index in range(n_ringdowns):
        result_file=path / "analysis" / f"index_{index}" / "result.hdf5"
        try:
            with h5py.File(result_file, "r") as res_file:
                log_z[key][index] = res_file["log_evidence"][()]
                info[key][index] = res_file["information"][()]
        except OSError:
            log_z[key][index] = np.nan
            info[key][index] = np.ana

In [ ]:
freqs = np.empty(n_ringdowns)
for index in range(n_ringdowns):
    freqs[index] = get_frequency(data_file, index)

In [ ]:
markers = {
    "simple_model_constant_noise": "o",
    "simple_model_amp_noise": "o",
    "terms_1": "s",
    "terms_3": "s",
    "terms_5": "s",
    "terms_7": "s",
    "terms_1_no_gap": "^", 
    "terms_3_no_gap": "^",
    "terms_5_no_gap": "^",
    "terms_7_no_gap": "^",
}
colours = {
    "simple_model_constant_noise": "k",
    "simple_model_amp_noise": "grey",
    "terms_1": "C0",
    "terms_3": "C1",
    "terms_5": "C2",
    "terms_7": "C3",
    "terms_1_no_gap": "C0", 
    "terms_3_no_gap": "C1",
    "terms_5_no_gap": "C2",
    "terms_7_no_gap": "C3",
}

In [ ]:
labels = {
    "simple_model_constant_noise": r"$M_1$, $\xi_2=0$",
    "simple_model_amp_noise": r"$M_1$, $\xi_2 \in [0, 1]$",
    "terms_1": r"$M_3$, $T=1$",
    "terms_3": r"$M_3$, $T=3$",
    "terms_5": r"$M_3$, $T=5$",
    "terms_7": r"$M_3$, $T=7$",
    "terms_1_no_gap": r"$M_3$, $T=1$, NG", 
    "terms_3_no_gap": r"$M_3$, $T=3$, NG",
    "terms_5_no_gap": r"$M_3$, $T=5$, NG",
    "terms_7_no_gap": r"$M_3$, $T=7$, NG",
}

In [ ]:
data_file = "../data/PyTotalAnalysis_2024_02_23.mat"
n_ringdowns = get_n_entries(data_file)

In [ ]:
frequencies = np.empty(n_ringdowns)
for index in range(n_ringdowns):
    frequencies[index] = get_frequency(data_file, index)

In [ ]:

durations = np.empty(n_ringdowns)
for index in range(n_ringdowns):
    durations[index] = get_duration(data_file, index)

In [ ]:
fig, axs = plt.subplots()
plot = {"terms_7"}
for (key, evals), ts in zip(nevals.items(), times.values()):
    if key not in plot:
        continue
    y_values = durations#parameters[key]["a_1"]
    # y_values = parameters[key]["a_1"] 
    # y_values = np.arange(len(evals))
    x_values = log_z[key]
    # y_values = parameters[key]["domega"]
    scatter = axs.scatter(x_values, y_values, label=key, marker=markers[key], s=10, c=ts / 60)
    # break
plt.colorbar(scatter, ax=axs, label="Wall time [mins]")
# axs.set_xscale("log")
# axs.set_yscale("log")
axs.set_xlabel(r"$\ln \cal{Z}$")
axs.set_ylabel(r"$t_{\text{Data}}$ [s]")
plt.tight_layout()
fig.savefig(outdir / f"time_dependence.{file_format}")
plt.show()

In [ ]:
figsize = plt.rcParams["figure.figsize"].copy()
figsize[1] *= 1.2
fig, axs = plt.subplots(figsize=figsize)
for (key, evals), ts in zip(nevals.items(), times.values()):
    plt.scatter(ts / 60, evals, label=labels[key], marker=markers[key], s=20 * np.abs(log_z[key] / log_z[key].max()), color=colours[key])
axs.set_xscale("log")
fig.legend(loc="center", bbox_to_anchor=(1.12, 0.55))
axs.set_yscale("log")
axs.set_xlabel("Wall time [mins]")
axs.set_ylabel("# likelihood evaluations")
plt.tight_layout()
fig.savefig(outdir / f"cost.{file_format}")
plt.show()